In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Amazon_Unlocked_Mobile.csv')
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [ ]:
df1 = df.drop(['Product Name', 'Brand Name', 'Price', 'Review Votes'], axis=1)
df1['label'] = df1['Rating'].apply(lambda c: 1 if c >3 else (0 if c<3 else 'neut'))
df1 = df1.drop(['Rating'], axis=1)
df1 = df1[df1['label'] != 'neut']
df1 = df1.replace(np.nan, '', regex=True)

In [ ]:
def get_top_data(top_n):
    top_data_df_positive = df1[df1['label'] == 1].head(top_n)
    top_data_df_negative = df1[df1['label'] == 0].head(top_n)
    top_data_df_small = pd.concat([top_data_df_positive, top_data_df_negative])
    return top_data_df_small

In [ ]:
# Function call to get the top 30000 from each sentiment
df1 = get_top_data(top_n=30000)

# After selecting top few samples of each sentiment
print("After segregating and taking equal number of rows for each sentiment:")
print(df1['label'].value_counts())
df1.head(10)

After segregating and taking equal number of rows for each sentiment:
1    30000
0    30000
Name: label, dtype: int64


,Reviews,label
0,I feel so LUCKY to have found this used (phone...,1
1,"nice phone, nice up grade from my pantach revu...",1
2,Very pleased,1
3,It works good but it goes slow sometimes but i...,1
4,Great phone to replace my lost phone. The only...,1
8,I originally was using the Samsung S2 Galaxy f...,1
11,This is a great product it came after two days...,1
12,These guys are the best! I had a little situat...,1
14,Ordered this phone as a replacement for the sa...,1
16,I was able to get the phone I previously owned...,1


In [ ]:
df1 = df1.sample(frac = 1)
df1.head()

,Reviews,label
35958,So far so good. Amazing.,1
100456,one phone do not work properly,0
60083,Save your money . You'll only get like a month...,0
85451,Keepon freezing its a refurbish phone n it say...,0
22371,"The phones appearance was fresh and new, but t...",0


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
df1['Reviews'] = df1['Reviews'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df1.head()

,Reviews,label
35958,so far so good. amazing.,1
100456,one phone do not work properly,0
60083,save your money . you'll only get like a month...,0
85451,keepon freezing its a refurbish phone n it say...,0
22371,"the phones appearance was fresh and new, but t...",0


In [ ]:
df1['Reviews'] = df1['Reviews'].str.replace('[^\w\s]','')
df1.head()

,Reviews,label
35958,so far so good amazing,1
100456,one phone do not work properly,0
60083,save your money youll only get like a months ...,0
85451,keepon freezing its a refurbish phone n it say...,0
22371,the phones appearance was fresh and new but th...,0


In [ ]:
stop = stopwords.words('english')
df1['Reviews'] = df1['Reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df1.head()

,Reviews,label
35958,far good amazing,1
100456,one phone work properly,0
60083,save money youll get like months worth usage s...,0
85451,keepon freezing refurbish phone n says new,0
22371,phones appearance fresh new operating system o...,0


In [ ]:
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
df1['Reviews']= df1['Reviews'].apply(space)

In [ ]:
df1.head()

,Reviews,label
35958,far good amazing,1
100456,one phone work properly,0
60083,save money -PRON- will get like month worth us...,0
85451,keepon freeze refurbish phone n say new,0
22371,phone appearance fresh new operating system op...,0


In [ ]:
X = df1.Reviews.values
Y = df1.label.values

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=32)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
X_train = vectorizer.transform(x_train)
X_test= vectorizer.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print("Accuracy: ", accuracy_score(y_test, predictions))

[[5700  320]
 [ 330 5650]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      6020
           1       0.95      0.94      0.95      5980

    accuracy                           0.95     12000
   macro avg       0.95      0.95      0.95     12000
weighted avg       0.95      0.95      0.95     12000

Accuracy:  0.9458333333333333
